<a href="https://colab.research.google.com/github/angelohafner/linguagem-de-programacao-udesc/blob/main/Reservoir_Simulation_Educational_Petroleum_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
"""
Petroleum Engineering Mini-System (Educational)
-----------------------------------------------
This single-file Python program models a very simplified oil reservoir production
scenario with multiple classes: Reservoir, Well, Fluid, MaterialBalance, and
ProductionSimulator. It is designed for instructional purposes.

All comments and docstrings are in English (teacher preference).
Method and attribute names use English. No compound operators like "+=" were used.

Units (consistent SI unless stated otherwise):
- Length: meters (m)
- Area: square meters (m^2)
- Thickness: meters (m)
- Pressure: kilopascals (kPa)
- Viscosity: centipoise (cP)
- Flow rate: cubic meters per day (m^3/day)
- Formation volume factor Bo: reservoir m^3 per stock-tank m^3 (rm3/sm3), unitless
- Time: days
- Permeability: millidarcies (mD), internally converted to m^2 when needed
- Compressibility (oil + rock total): 1/kPa

This code prioritizes clarity over realism and omits many real-world complexities.
"""

import math
from typing import List, Tuple


class Fluid:
    """
    Represents a simple oil fluid with minimal PVT properties.

    Attributes:
        oil_api (float): API gravity of the oil.
        viscosity_cp (float): Dynamic viscosity in cP (centipoise).
        formation_volume_factor_Bo (float): Bo, reservoir m^3 per stock-tank m^3 (rm3/sm3).
        reference_density_kg_m3 (float): Optional base density at stock-tank conditions, kg/m^3.

    Notes:
        - Density estimation here is simplistic and uses API to estimate stock-tank density.
        - Real PVT behavior is not modeled (no pressure dependence for Bo or viscosity).
    """

    def __init__(
        self,
        oil_api: float,
        viscosity_cp: float,
        formation_volume_factor_Bo: float,
        reference_density_kg_m3: float = None,
    ):
        self.oil_api = oil_api
        self.viscosity_cp = viscosity_cp
        self.formation_volume_factor_Bo = formation_volume_factor_Bo
        if reference_density_kg_m3 is None:
            # Estimate stock-tank oil density from API gravity (very rough):
            # rho_oil (kg/m3) ≈ 141.5 / (API + 131.5) * 999.1
            self.reference_density_kg_m3 = (
                141.5 / (self.oil_api + 131.5) * 999.1
            )
        else:
            self.reference_density_kg_m3 = reference_density_kg_m3

    def density_stock_tank(self) -> float:
        """
        Returns:
            float: Estimated stock-tank oil density in kg/m^3.
        """
        return self.reference_density_kg_m3


class Reservoir:
    """
    Represents a very simplified homogeneous reservoir.

    Attributes:
        area_m2 (float): Areal extent (m^2).
        thickness_m (float): Net pay (m).
        porosity (float): Fraction (0 to 1).
        initial_pressure_kpa (float): Initial pressure (kPa).
        swi (float): Initial water saturation (fraction).
        permeability_md (float): Permeability (mD).
        outer_radius_m (float): External drainage radius for radial flow (m).
        total_compressibility_per_kpa (float): Total compressibility c_t (1/kPa), rock+fluid.

    Methods:
        estimate_OOIP_stm3(fluid): Volumetric OOIP using A*h*phi*(1 - Swi) / Bo.
        pore_volume_m3(): A*h*phi.
        update_pressure_by_production(delta_Np_stm3, fluid): Simple MBE-like pressure drop.
    """

    def __init__(
        self,
        area_m2: float,
        thickness_m: float,
        porosity: float,
        initial_pressure_kpa: float,
        swi: float,
        permeability_md: float,
        outer_radius_m: float,
        total_compressibility_per_kpa: float,
    ):
        self.area_m2 = area_m2
        self.thickness_m = thickness_m
        self.porosity = porosity
        self.initial_pressure_kpa = initial_pressure_kpa
        self.current_pressure_kpa = initial_pressure_kpa
        self.swi = swi
        self.permeability_md = permeability_md
        self.outer_radius_m = outer_radius_m
        self.total_compressibility_per_kpa = total_compressibility_per_kpa

    def pore_volume_m3(self) -> float:
        """
        Returns:
            float: Pore volume PV = A * h * phi (m^3).
        """
        pv = self.area_m2 * self.thickness_m * self.porosity
        return pv

    def estimate_OOIP_stm3(self, fluid: Fluid) -> float:
        """
        Estimates OOIP (stock-tank m^3) using a simple volumetric formula:
            OOIP ≈ (A * h * phi * (1 - Swi)) / Bo

        Args:
            fluid (Fluid): Fluid instance with Bo.

        Returns:
            float: OOIP in stock-tank m^3.
        """
        bulk_oil_reservoir_m3 = self.area_m2 * self.thickness_m * self.porosity * (1.0 - self.swi)
        if fluid.formation_volume_factor_Bo <= 0.0:
            raise ValueError("Bo must be positive.")
        ooip = bulk_oil_reservoir_m3 / fluid.formation_volume_factor_Bo
        return ooip

    def update_pressure_by_production(self, delta_Np_stm3: float, fluid: Fluid) -> None:
        """
        Updates current reservoir pressure using a simple linearized material-balance-like relation:
            dP ≈ (Np_increment * Bo) / (c_t * PV)

        Args:
            delta_Np_stm3 (float): Incremental stock-tank oil produced in this step (m^3).
            fluid (Fluid): Fluid instance (uses Bo).

        Notes:
            - Very simplified; ignores gas liberation, water influx coupling, etc.
            - Assumes undersaturated oil with constant properties.
        """
        pv = self.pore_volume_m3()
        ct = self.total_compressibility_per_kpa
        bo = fluid.formation_volume_factor_Bo
        if pv <= 0.0 or ct <= 0.0 or bo <= 0.0:
            return
        dP = (delta_Np_stm3 * bo) / (ct * pv)
        new_pressure = self.current_pressure_kpa - dP
        if new_pressure < 0.0:
            new_pressure = 0.0
        self.current_pressure_kpa = new_pressure


class Well:
    """
    Represents a vertical well with simple radial flow behavior.

    Attributes:
        depth_m (float): Well depth (m).
        perforation_radius_m (float): Wellbore radius (m).
        skin_factor (float): Skin factor (dimensionless).
        bottomhole_pressure_kpa (float): Flowing bottom-hole pressure (kPa), may be fixed.

    Methods:
        calculate_flow_rate(pressure_reservoir_kpa, reservoir, fluid): Computes q (m^3/day).
    """

    def __init__(
        self,
        depth_m: float,
        perforation_radius_m: float,
        skin_factor: float,
        bottomhole_pressure_kpa: float,
    ):
        self.depth_m = depth_m
        self.perforation_radius_m = perforation_radius_m
        self.skin_factor = skin_factor
        self.bottomhole_pressure_kpa = bottomhole_pressure_kpa

    @staticmethod
    def md_to_m2(k_md: float) -> float:
        """
        Converts permeability in millidarcies to square meters.
        1 Darcy ≈ 9.869233e-13 m^2; 1 mD = 1e-3 Darcy.
        """
        return k_md * 1.0e-3 * 9.869233e-13

    def calculate_flow_rate(
        self,
        pressure_reservoir_kpa: float,
        reservoir: Reservoir,
        fluid: Fluid,
    ) -> float:
        """
        Computes a simplified radial Darcy flow for a vertical well (single-phase oil):

        q = (2 * pi * k * h * (Pr - Pw)) / (mu * B * (ln(re/rw) + s)) * C

        Where:
            - q is m^3/day (stock-tank basis approximately)
            - k in m^2, h in m, pressures in Pa (converted from kPa)
            - mu in Pa·s (converted from cP: 1 cP = 1e-3 Pa·s)
            - B is Bo (rm3/sm3)
            - re is outer drainage radius, rw is well radius
            - s is skin factor
            - C is a unit conversion factor to get to day and volumetric basis

        Returns:
            float: Flow rate in m^3/day (approximate stock-tank basis).
        """
        pr_kpa = pressure_reservoir_kpa
        pwf_kpa = self.bottomhole_pressure_kpa
        if pr_kpa <= pwf_kpa:
            return 0.0

        k_m2 = Well.md_to_m2(reservoir.permeability_md)
        h_m = reservoir.thickness_m
        re_m = reservoir.outer_radius_m
        rw_m = self.perforation_radius_m
        s = self.skin_factor
        mu_pas = fluid.viscosity_cp * 1.0e-3  # cP to Pa·s
        bo = fluid.formation_volume_factor_Bo

        if re_m <= rw_m:
            re_m = rw_m * 10.0

        delta_p_pa = (pr_kpa - pwf_kpa) * 1000.0  # kPa to Pa

        denom = math.log(re_m / rw_m) + s
        if denom <= 0.0:
            denom = 1.0

        # Base radial Darcy (SI): q_reservoir_m3_per_s
        q_res_m3_per_s = (2.0 * math.pi * k_m2 * h_m * delta_p_pa) / (mu_pas * bo * denom)

        # Convert to per day
        q_res_m3_per_day = q_res_m3_per_s * 86400.0

        # We will treat this as stock-tank m^3/day approximately (already divided by Bo).
        if q_res_m3_per_day < 0.0:
            q_res_m3_per_day = 0.0
        return q_res_m3_per_day


class MaterialBalance:
    """
    Simple production accounting and placeholder drive mechanisms.

    Tracks:
        cumulative_production_stm3 (float)
        time_days (List[float])
        rates_m3_per_day (List[float])
        pressures_kpa (List[float])

    Methods:
        record_step(t, q, p): Store time series.
        compute_drive_index(): Dummy partition of drive mechanisms.
        estimate_water_influx(): Placeholder returning zero.
    """

    def __init__(self):
        self.cumulative_production_stm3 = 0.0
        self.time_days: List[float] = []
        self.rates_m3_per_day: List[float] = []
        self.pressures_kpa: List[float] = []

    def record_step(self, t_days: float, q_m3_day: float, p_kpa: float, dt_days: float) -> None:
        """
        Records time, rate, pressure and updates cumulative production.

        Args:
            t_days (float): Current time (days).
            q_m3_day (float): Flow rate (m^3/day).
            p_kpa (float): Reservoir pressure (kPa).
            dt_days (float): Time step (days).
        """
        produced_increment = q_m3_day * dt_days
        self.cumulative_production_stm3 = self.cumulative_production_stm3 + produced_increment
        self.time_days.append(t_days)
        self.rates_m3_per_day.append(q_m3_day)
        self.pressures_kpa.append(p_kpa)

    def compute_drive_index(self) -> Tuple[float, float, float]:
        """
        Returns:
            tuple: (expansion_drive, water_drive, gas_cap_drive), summing to 1.0.
        Notes:
            - Placeholder: assigns majority to expansion.
        """
        expansion = 0.8
        water = 0.2
        gas_cap = 0.0
        return (expansion, water, gas_cap)

    def estimate_water_influx(self, p_kpa: float, t_days: float) -> float:
        """
        Placeholder water influx model.

        Returns:
            float: Estimated water influx rate (m^3/day), here always zero.
        """
        return 0.0


class ProductionSimulator:
    """
    Orchestrates a simple production simulation with fixed time steps.

    Attributes:
        reservoir (Reservoir)
        well (Well)
        fluid (Fluid)
        mb (MaterialBalance)

    Methods:
        run(duration_days, dt_days): Simulates production, returns a list of (t, p, q, Np).
        print_table(results): Prints a formatted table to the console.
    """

    def __init__(self, reservoir: Reservoir, well: Well, fluid: Fluid):
        self.reservoir = reservoir
        self.well = well
        self.fluid = fluid
        self.mb = MaterialBalance()

    def run(self, duration_days: float, dt_days: float) -> List[Tuple[float, float, float, float]]:
        """
        Runs the time-stepping simulation.

        Args:
            duration_days (float): Total time in days.
            dt_days (float): Time step in days.

        Returns:
            List of tuples: (time_days, pressure_kpa, flow_rate_m3_day, cumulative_stm3)
        """
        results: List[Tuple[float, float, float, float]] = []
        t = 0.0
        while t <= duration_days:
            p = self.reservoir.current_pressure_kpa
            q = self.well.calculate_flow_rate(
                pressure_reservoir_kpa=p,
                reservoir=self.reservoir,
                fluid=self.fluid,
            )

            self.mb.record_step(t_days=t, q_m3_day=q, p_kpa=p, dt_days=dt_days)
            self.reservoir.update_pressure_by_production(delta_Np_stm3=q * dt_days, fluid=self.fluid)

            results.append((t, p, q, self.mb.cumulative_production_stm3))
            t = t + dt_days
        return results

    @staticmethod
    def print_table(results: List[Tuple[float, float, float, float]]) -> None:
        """
        Prints a simple table: time, pressure, rate, cumulative production.
        """
        header = (
            " Time (d) | Pressure (kPa) | Rate (m3/d) | Cum Prod (m3) "
        )
        line = "-" * len(header)
        print(line)
        print(header)
        print(line)
        for row in results:
            t, p, q, np_cum = row
            print(
                f" {t:8.2f} | {p:14.1f} | {q:10.3f} | {np_cum:12.3f}"
            )
        print(line)


def generate_sample_case() -> Tuple[Reservoir, Well, Fluid]:
    """
    Builds a ready-to-run sample case with plausible but simple numbers.

    Returns:
        tuple: (reservoir, well, fluid)
    """
    # Fluid: light oil, moderate viscosity, modest Bo
    fluid = Fluid(
        oil_api=35.0,
        viscosity_cp=2.0,
        formation_volume_factor_Bo=1.2,
    )

    # Reservoir: moderate size, moderate permeability
    reservoir = Reservoir(
        area_m2=1.0e6,                     # ~1 km^2
        thickness_m=20.0,                  # 20 m net pay
        porosity=0.20,                     # 20%
        initial_pressure_kpa=25000.0,      # 25 MPa
        swi=0.25,                          # 25%
        permeability_md=200.0,             # 200 mD
        outer_radius_m=500.0,              # 500 m drainage radius
        total_compressibility_per_kpa=1.0e-5,  # 1e-5 1/kPa (rock + fluid)
    )

    # Well: small radius, mild positive skin, fixed bottom-hole pressure
    well = Well(
        depth_m=2500.0,
        perforation_radius_m=0.1,          # 10 cm radius
        skin_factor=2.0,
        bottomhole_pressure_kpa=15000.0,   # 15 MPa flowing BHP
    )

    return reservoir, well, fluid


def main() -> None:
    """
    Runs the sample simulation and prints outputs.
    Also executes simple consistency assertions.
    """
    reservoir, well, fluid = generate_sample_case()

    # Report OOIP estimate
    ooip_stm3 = reservoir.estimate_OOIP_stm3(fluid)
    print("Estimated OOIP (stock-tank m^3):", f"{ooip_stm3:,.0f}")

    sim = ProductionSimulator(reservoir, well, fluid)
    duration_days = 100.0
    dt_days = 5.0
    results = sim.run(duration_days=duration_days, dt_days=dt_days)
    sim.print_table(results)

    # Simple assertions for sanity checks (didactic only)
    # 1) Cumulative production must be non-decreasing
    last_np = -1.0
    for _, _, _, np_cum in results:
        assert np_cum >= last_np, "Cumulative production should not decrease."
        last_np = np_cum

    # 2) Pressure should not increase under production
    pressures = [p for _, p, _, _ in results]
    for i in range(1, len(pressures)):
        assert pressures[i] <= pressures[i - 1] + 1e-6, "Pressure should be non-increasing."

    # 3) Rate should eventually decline as pressure declines (not guaranteed monotone each step)
    #    We check that final rate is lower than initial rate (weak check).
    initial_rate = results[0][2]
    final_rate = results[-1][2]
    assert final_rate <= initial_rate + 1e-9, "Final rate should be less than or equal to initial rate."

    # Print comprehension questions (without answers) by default
    SHOW_SOLUTIONS = False
    print("\n--- Comprehension Questions (give only this block to students) ---\n")
    print(QUESTIONS_TEXT)

    if SHOW_SOLUTIONS:
        print("\n--- Suggested Answers (teacher only) ---\n")
        print(ANSWERS_TEXT)

    # End of main.


# -------------------------
# Pedagogical material
# -------------------------

QUESTIONS_TEXT = """
1) Explique como o método Reservoir.estimate_OOIP_stm3 funciona e liste suas principais premissas.
2) No método Well.calculate_flow_rate, quais termos controlam a sensibilidade da taxa de produção em relação à depressão de pressão?
3) Por que o denominador (ln(re/rw) + s) está presente na fórmula de fluxo radial de Darcy? Qual é o significado físico do fator de dano (skin) s?
4) Qual é o papel do fator de volume de formação Bo neste código? Como ele relaciona os volumes do reservatório e de tanque de superfície?
5) O método Reservoir.update_pressure_by_production usa uma relação simplificada de balanço de materiais. Quais variáveis determinam a queda de pressão por passo de tempo?
6) Por que a taxa de produção pode diminuir ao longo do tempo mesmo que a pressão de fundo (bottom-hole pressure) seja mantida constante?
7) O que é o volume poroso (PV) e como ele é calculado aqui? Por que o PV é importante para a queda de pressão?
8) Quais suposições neste código deixariam de ser válidas em um reservatório de óleo saturado com liberação de gás?
9) O método MaterialBalance.compute_drive_index retorna (0.8, 0.2, 0.0). Em um caso real, quais observações poderiam alterar essas frações?
10) Proponha uma melhoria para tornar o modelo mais realista sem adicionar bibliotecas externas.
"""


EXTENSION_TASKS = """
Tarefas de Extensão (para alunos avançados)
A) Implemente um modelo simples de influxo de aquífero (por exemplo, Fetkovich) e compare os históricos de pressão/taxa.
B) Substitua a viscosidade e o Bo fixos por correlações dependentes da pressão; execute análises de sensibilidade.
C) Adicione uma etapa de análise nodal: calcule a entregabilidade do poço com IPR e desempenho da coluna de produção para determinar Pw (em vez de BHP fixo).
"""


if __name__ == "__main__":
    main()

    # The following block is provided so the instructor can easily access and copy extension tasks.
    print("\n--- Extension Tasks (advanced students) ---\n")
    print(EXTENSION_TASKS)


Estimated OOIP (stock-tank m^3): 2,500,000
---------------------------------------------------------
 Time (d) | Pressure (kPa) | Rate (m3/d) | Cum Prod (m3) 
---------------------------------------------------------
     0.00 |        25000.0 |    849.036 |     4245.178
     5.00 |        24872.6 |    838.223 |     8436.292
    10.00 |        24746.9 |    827.547 |    12574.029
    15.00 |        24622.8 |    817.008 |    16659.070
    20.00 |        24500.2 |    806.603 |    20692.086
    25.00 |        24379.2 |    796.331 |    24673.739
    30.00 |        24259.8 |    786.189 |    28604.684
    35.00 |        24141.9 |    776.176 |    32485.566
    40.00 |        24025.4 |    766.291 |    36317.023
    45.00 |        23910.5 |    756.532 |    40099.685
    50.00 |        23797.0 |    746.897 |    43834.172
    55.00 |        23685.0 |    737.385 |    47521.099
    60.00 |        23574.4 |    727.994 |    51161.070
    65.00 |        23465.2 |    718.723 |    54754.685
    70.00 |  